In [ ]:
import pandas as pd

import sys
import os
sys.path.append(os.path.abspath('../code'))

import config
import re
from util import Util

l = 0
d = {k: [] for k in ["pap-simple", "nl-simple"]}
filename = config.rblx["rawdatafile"]

reading_pap = False
cur_nl_words = []

with open(f"{filename}", encoding="UTF-8") as fileobject:
    for line in fileobject:
        l += 1
        if l < config.rblx["startline"] or l > config.rblx["endline"]:
            continue
        ab = re.compile(config.rblx["sectionseparator"])
        if ab.match(line):
            print(f"Skipping {line}")
            continue

        line = Util.remove_tags(line)
        # Deal with space around tags
        line = " ".join(line.split())
        line = line.strip()
        
        line = Util.removeTrailingPunctuation(line)

        # print(line)

        if "," in line:
            words = line.split(",")
        else:
            words = [line]
        if reading_pap:
            print(f"pap:{words} nl:{cur_nl_words}")        
            for pap_word in words:
                for nl_word in cur_nl_words:
                    d["pap-simple"].append(pap_word.strip())
                    d["nl-simple"].append(nl_word.strip())
        else:
            cur_nl_words = words

        reading_pap = not reading_pap

df = pd.DataFrame.from_dict(d)
print(f"final {df}")

In [ ]:
df.to_csv(config.rblx["outfile"], index=None)